In [1]:
pip install kaggle

In [2]:
! pip install kaggle

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [6]:
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 91% 74.0M/80.9M [00:00<00:00, 101MB/s] 
100% 80.9M/80.9M [00:01<00:00, 81.9MB/s]


In [7]:
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


In [8]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [9]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [10]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [11]:
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding = 'ISO-8859-1')


In [12]:
twitter_data.shape

(1599999, 6)

In [14]:
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [15]:
column_name=['target','ID','date','flag','user','text']
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', names=column_name ,encoding = 'ISO-8859-1')


In [16]:
twitter_data.shape

(1600000, 6)

In [17]:
twitter_data.head()

,target,ID,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [18]:
twitter_data.isnull().sum()

target    0
ID        0
date      0
flag      0
user      0
text      0
dtype: int64

In [19]:
twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [20]:
twitter_data.replace({'target' :{4:1}}, inplace=True)

In [21]:
twitter_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

In [22]:
stemmer = PorterStemmer()

In [24]:
#Creating a function for stemming like a templet
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content) #content is the tweet, ^ means removing all the things that dont come in the alphabetical range, like puctuation marks etc.
    stemmed_content = content.lower() #Converting into lower case
    stemmed_content = stemmed_content.split() #Splitting the words into list
    stemmed_content = [stemmer.stem(word) for word in stemmed_content if not word in stopwords.words('english')] #stemming and removing stopwords
    stemmed_content = ' '.join(stemmed_content) #joining the list into string in every single tweet
    return stemmed_content #returning the stemmed content

In [25]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [26]:
twitter_data.head()

,target,ID,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","@switchfoot http://twitpic.com/2y1zl - awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset can't updat facebook text it... might cr...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,@kenichan dive mani time ball. manag save 50% ...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....","@nationwideclass no, behav all. i'm mad. here?..."


In [27]:
print(twitter_data['stemmed_content'])

0          @switchfoot http://twitpic.com/2y1zl - awww, t...
1          upset can't updat facebook text it... might cr...
2          @kenichan dive mani time ball. manag save 50% ...
3                            whole bodi feel itchi like fire
4          @nationwideclass no, behav all. i'm mad. here?...
                                 ...                        
1599995                       woke up. school best feel ever
1599996    thewdb.com - cool hear old walt interviews! â...
1599997                      readi mojo makeover? ask detail
1599998    happi 38th birthday boo alll time!!! tupac ama...
1599999    happi #charitytuesday @thenspcc @sparkschar @s...
Name: stemmed_content, Length: 1600000, dtype: object


In [28]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [29]:
#Seperating data and label
X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values


In [30]:
print(X)


["@switchfoot http://twitpic.com/2y1zl - awww, that' bummer. shoulda got david carr third day it. ;d"
 "upset can't updat facebook text it... might cri result school today also. blah!"
 '@kenichan dive mani time ball. manag save 50% rest go bound' ...
 'readi mojo makeover? ask detail'
 'happi 38th birthday boo alll time!!! tupac amaru shakur'
 'happi #charitytuesday @thenspcc @sparkschar @speakinguph4h']


In [31]:
print(Y)

[0 0 0 ... 1 1 1]


In [32]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [33]:
print(X.shape, X_train.shape, X_test.shape)

(1600000,) (1280000,) (320000,)


In [34]:
print(X_train)

['watch saw iv drink lil wine' "@hatermagazin i'm in!"
 'even though favourit drink think vodka coke wipe mind time think im gonna find new drink'
 ... 'eager monday afternoon'
 "hope everyon mother great day! can't wait hear guy store tomorrow!"
 'love wake folgers. bad voic deeper his.']


In [35]:
print(X_test)

['@mmangen - fine! much time chat twitter. hubbi back summer &amp; tend domin free time.'
 'ah may show w/ ruth kim &amp; geoffrey sanhueza'
 '@ishatara mayb bay area thang dammit!' ...
 '@destini41 nevertheless hooray! 4700 member wonder safe trip!'
 'feel well' '@supersandro !!!! thank you!']


In [36]:
#converting textual data into numericla data
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train) #fitting the vectorizer to understand the data and transforming data to give a score to the data coverting it into numerical value
X_test = vectorizer.transform(X_test) #transform accord to the train data

In [37]:
print(X_train)

  (0, 555967)	0.448720931098355
  (0, 311959)	0.4213989894780769
  (0, 166830)	0.37842601975344053
  (0, 254267)	0.5248773503338933
  (0, 453116)	0.35795997954657904
  (0, 548734)	0.272460245511403
  (1, 247278)	0.4216951552804088
  (1, 226262)	0.9067376665899747
  (2, 375364)	0.1668911233829414
  (2, 193769)	0.20184466588151023
  (2, 214788)	0.18696286991487276
  (2, 245846)	0.16108271281506448
  (2, 516778)	0.15260594669966604
  (2, 349418)	0.2408559466227688
  (2, 556376)	0.33614580595161764
  (2, 129252)	0.3117944515681846
  (2, 545128)	0.32796751926832934
  (2, 513655)	0.3201716231105769
  (2, 190522)	0.29198833954883197
  (2, 514638)	0.18602125106063971
  (2, 184426)	0.1903000016055046
  (2, 166830)	0.46135196432581554
  (3, 239574)	0.38340980743118147
  (3, 519470)	0.2688525366668751
  (3, 493022)	0.3914375454549087
  :	:
  (1279996, 410084)	0.21029847974488325
  (1279996, 315358)	0.2219434776579764
  (1279996, 378383)	0.1776339096901187
  (1279996, 520651)	0.1868087345439146
  

In [38]:
print(X_test)

  (0, 530649)	0.17887663933344994
  (0, 516778)	0.31514526424889533
  (0, 505258)	0.3459496612358391
  (0, 492691)	0.21841828884993894
  (0, 364065)	0.17667781375470604
  (0, 354509)	0.449467530296984
  (0, 238369)	0.2869441563378548
  (0, 200572)	0.23596147172673212
  (0, 193829)	0.2532904390317208
  (0, 163021)	0.37372158998727745
  (0, 117476)	0.26691751430897404
  (0, 74691)	0.1610538837848407
  (0, 55622)	0.17041370659097008
  (1, 466635)	0.24404319903374125
  (1, 445343)	0.48174286114843634
  (1, 337825)	0.2840187026690571
  (1, 291622)	0.4070550884371154
  (1, 208893)	0.5712382041014992
  (1, 55622)	0.20816041520057577
  (1, 45607)	0.3039118621705182
  (2, 507224)	0.4337196481249129
  (2, 337915)	0.2587039123196698
  (2, 252255)	0.5874470296410402
  (2, 143688)	0.3624696061947235
  (2, 78469)	0.3786845038674409
  :	:
  (319995, 526794)	0.39930162664337965
  (319995, 417479)	0.3357981149453034
  (319995, 407164)	0.2582052217116053
  (319995, 384821)	0.40236048960872395
  (319995,

In [39]:
model = LogisticRegression(max_iter=1000) #max_iter means how many times machine goes through data

In [40]:
model.fit(X_train, Y_train)


LogisticRegression(max_iter=1000)

In [41]:
#Accuracy score on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)

In [42]:
print('Accuracy score on the training data :', training_data_accuracy)


Accuracy score on the training data : 0.8169171875


In [43]:
#Accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)

In [44]:
print('Accuracy score on the training data :', test_data_accuracy)

Accuracy score on the training data : 0.782253125


In [45]:
import pickle

In [46]:
filename = 'trained_model.sav'
pickle.dump(model, open(filename,'wb'))

In [47]:
#Loading the saved Model
loaded_model = pickle.load(open('/content/trained_model.sav', 'rb')) #rb means read binary


In [48]:
X_new = X_test[200]
print(Y_test[200])
prediction = loaded_model.predict(X_new)
print(prediction)
if(prediction[0]==0):
 print('Negative tweet')
else:
 print('Positive tweet')

1
[1]
Positive tweet


In [49]:
X_new = X_test[3]
print(Y_test[3])
prediction = loaded_model.predict(X_new)
print(prediction)
if(prediction[0]==0):
 print('Negative tweet')
else:
 print('Positive tweet')

0
[0]
Negative tweet


In [50]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 76.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 28.2 MB

In [51]:
import gradio as gr

In [52]:
def predict_sentiment(X_test):
    X_test = vectorizer.transform([X_test])
    prediction = model.predict(X_test)
    return 'Positive' if prediction == 1 else 'Negative'
X_test1 = input()
print(f'Sentiment: {predict_sentiment(X_test1)}')

happy
Sentiment: Positive


In [53]:
interface = gr.Interface(fn =predict_sentiment, inputs = 'text', outputs = 'text' )

In [54]:
interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://2fa8412a7ae07cc995.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
